## Basic Multi-LLM Workflows

이 노트북은 3가지 기본적인 Multi-LLM 워크플로우를 소개 합니다. 각각의 워크플로우는 비용이나 지연시간과 작업 성능 향상 사이의 균형을 고려한 방식 입니다:

1. **Prompt-Chaining**: 하나의 작업을 순차적은 하위 작업들로 분해하여, 각 단계가 이전 결과를 기반으로 수행됩니다.
2. **Parallelization**: 독립적인 하위 작업들을 여러 LLM에 분산하여 동시에 처리 합니다.
3. **Routing**: 입력 특성에 따라 특화된 LLM 경로를 동적으로 선택합니다.

참고: 이 구현 예시들은 핵심 개념을 설명하기 위한 것으로, 실제 프로덕션용 코드가 아닙니다.

In [1]:
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict, Callable
from util import llm_call, extract_xml

In [2]:
def chain(input: str, prompts: List[str]) -> str:
    """Chain multiple LLM calls sequentially, passing results between steps."""
    result = input
    for i, prompt in enumerate(prompts, 1):
        print(f"\nStep {i}:")
        result = llm_call(f"{prompt}\nInput: {result}")
        print(result)
    return result

def parallel(prompt: str, inputs: List[str], n_workers: int = 3) -> List[str]:
    """Process multiple inputs concurrently with the same prompt."""
    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        futures = [executor.submit(llm_call, f"{prompt}\nInput: {x}") for x in inputs]
        return [f.result() for f in futures]

def route(input: str, routes: Dict[str, str]) -> str:
    """Route input to specialized prompt using content classification."""
    print(f"\nAvailable routes: {list(routes.keys())}")
    selector_prompt = f"""
    Analyze the input and select the most appropriate support team from these options: {list(routes.keys())}
    First explain your reasoning, then provide your selection in this XML format:

    <reasoning>
    Brief explanation of why this ticket should be routed to a specific team.
    Consider key terms, user intent, and urgency level.
    </reasoning>

    <selection>
    The chosen team name
    </selection>

    Input: {input}""".strip()
    
    route_response = llm_call(selector_prompt)
    reasoning = extract_xml(route_response, 'reasoning')
    route_key = extract_xml(route_response, 'selection').strip().lower()
    
    print("Routing Analysis:")
    print(reasoning)
    print(f"\nSelected route: {route_key}")
    
    selected_prompt = routes[route_key]
    return llm_call(f"{selected_prompt}\nInput: {input}")


## 사용 예시

아래는 각 워크플로우를 보여주는 실제 예시들 입니다:
1. 구조화된 데이터 추출 및 형식화를 위한 <b>체인 워크플로우</b>
2. <b>병렬 처리 워크플로우</b>를 활용한 stakeholder 영향도 분석
3. 고객 지원 티켓 처리를 위한 <b>라우팅 워크플로우</b>

In [3]:
# 예시 1: 구조화된 데이터 추출 및 형식화를 위한 체인 워크플로우
# 각 단계는 raw 텍스트를 점진적으로 형식화된 표로 변환합니다

data_processing_steps = [
    """텍스트에서 숫자값과 관련 지표만 추출하세요.
    각각을 새로운 줄에 'value: metric' 형식으로 작성하세요.
    예시 형식:
    92: 고객 만족도
    45%: 매출 성장률""",
    
    """가능한 모든 숫자값을 백분율로 변환하세요.
    백분율이나 포인트가 아닌 경우 소수로 변환하세요 (예: 92 points -> 92%).
    한 줄당 하나의 숫자를 유지하세요.
    예시 형식:
    92%: 고객 만족도
    45%: 매출 성장률""",
    
    """모든 줄을 숫자값 기준 내림차순으로 정렬하세요.
    각 줄의 'value: metric' 형식을 유지하세요.
    예시:
    92%: 고객 만족도
    87%: 직원 만족도""",
    
    """정렬된 데이터를 다음과 같은 마크다운 표 형식으로 작성하세요:
    | Metric | Value |
    |:--|--:|
    | 고객 만족도 | 92% |"""
]

report = """
3분기 실적 요약:
이번 분기 고객 만족도 점수는 92포인트로 상승했습니다.
매출은 전년 대비 45% 성장했습니다.
주요 시장에서의 시장 점유율은 현재 23%입니다.
고객 이탈률은 8%에서 5%로 감소했습니다.
신규 사용자 획득 비용은 사용자당 $43입니다.
제품 도입률은 78%로 증가했습니다.
직원 만족도는 87포인트입니다.
영업이익률은 34%로 개선되었습니다.
"""

print("\nInput text:")
print(report)
formatted_result = chain(report, data_processing_steps)
print(formatted_result)


Input text:

3분기 실적 요약:
이번 분기 고객 만족도 점수는 92포인트로 상승했습니다.
매출은 전년 대비 45% 성장했습니다.
주요 시장에서의 시장 점유율은 현재 23%입니다.
고객 이탈률은 8%에서 5%로 감소했습니다.
신규 사용자 획득 비용은 사용자당 $43입니다.
제품 도입률은 78%로 증가했습니다.
직원 만족도는 87포인트입니다.
영업이익률은 34%로 개선되었습니다.


Step 1:
92: 고객 만족도
45%: 매출 성장률
23%: 시장 점유율
8%: 이전 고객 이탈률
5%: 현재 고객 이탈률
$43: 신규 사용자 획득 비용
78%: 제품 도입률
87: 직원 만족도
34%: 영업이익률

Step 2:
92%: 고객 만족도
45%: 매출 성장률
23%: 시장 점유율
8%: 이전 고객 이탈률
5%: 현재 고객 이탈률
43%: 신규 사용자 획득 비용
78%: 제품 도입률
87%: 직원 만족도
34%: 영업이익률

Step 3:
92%: 고객 만족도
87%: 직원 만족도
78%: 제품 도입률
45%: 매출 성장률
43%: 신규 사용자 획득 비용
34%: 영업이익률
23%: 시장 점유율
8%: 이전 고객 이탈률
5%: 현재 고객 이탈률

Step 4:
| Metric | Value |
|:--|--:|
| 고객 만족도 | 92% |
| 직원 만족도 | 87% |
| 제품 도입률 | 78% |
| 매출 성장률 | 45% |
| 신규 사용자 획득 비용 | 43% |
| 영업이익률 | 34% |
| 시장 점유율 | 23% |
| 이전 고객 이탈률 | 8% |
| 현재 고객 이탈률 | 5% |
| Metric | Value |
|:--|--:|
| 고객 만족도 | 92% |
| 직원 만족도 | 87% |
| 제품 도입률 | 78% |
| 매출 성장률 | 45% |
| 신규 사용자 획득 비용 | 43% |
| 영업이익률 | 34% |
| 시장 점유율 | 23% |
| 이전 고객 이탈률 | 8% |
| 현재 고객 이탈률 | 5% |


In [4]:
# 예시 2: 병렬 처리 워크플로우를 활용한 stakeholder 영향도 분석
# 여러 Stakeholder 그룹에 대한 영향 분석을 동시에 처리

stakeholders = [
    """고객:
    - 가격에 민감함
    - 더 나은 기술 원함
    - 환경 관련 우려사항""",
    
    """직원:
    - 고용 안정성 우려
    - 새로운 기술 습득 필요
    - 명확한 방향성 요구""",
    
    """투자자:
    - 성장 기대
    - 비용 통제 원함
    - 리스크 우려사항""",
    
    """공급업체:
    - 생산능력 제약
    - 가격 압박
    - 기술 전환"""
]

impact_results = parallel(
    """이 stakeholder 그룹에 대한 시장 변화의 영향을 분석하세요.
    구체적인 영향과 권장 조치를 제공하세요.
    명확한 섹션과 우선순위로 형식을 갖추세요.""",
    stakeholders
)

for result in impact_results:
    print(result)

# 고객 영향 분석 및 권장 조치

## 1. 가격 민감도 영향 (높은 우선순위)
영향:
- 경쟁 심화로 인한 가격 압박 증가
- 구매력 감소로 인한 수요 위축 가능성
- 가격 대비 가치 중요성 증가

권장 조치:
- 원가 절감을 통한 경쟁력 있는 가격 책정
- 가격대별 제품 라인업 다양화
- 할인/프로모션 프로그램 개발

## 2. 기술 요구사항 (중간 우선순위)
영향:
- 신기술 도입 비용 증가
- 제품 개발 주기 단축 필요성
- 경쟁사와의 기술격차 중요성 증가

권장 조치:
- R&D 투자 확대
- 사용자 친화적 기술 개발
- 기술 교육/지원 서비스 강화

## 3. 환경 관련 우려 (증가하는 우선순위)
영향:
- 친환경 제품 수요 증가
- 환경 규제 준수 비용 상승
- 기업 이미지에 영향

권장 조치:
- 친환경 제품 라인 확대
- 지속가능성 인증 획득
- 환경 정책 투명성 강화

## 모니터링 지표
- 고객 만족도
- 시장 점유율
- 환경 영향 평가
- 기술 혁신 지수
# 직원 영향 분석 및 권장 조치 보고서

## 1. 주요 영향 분석 (우선순위 순)

### 고용 안정성 우려 (High Priority)
- 구조조정 가능성으로 인한 불안감 증가
- 업무 범위 및 역할 변화에 대한 불확실성
- 장기 경력 개발 경로의 불명확성

### 새로운 기술 습득 필요 (Medium Priority)
- 디지털 전환에 따른 기술 격차 발생
- 적응 기간 동안의 생산성 저하 우려
- 학습 부담으로 인한 스트레스 증가

### 명확한 방향성 요구 (Medium Priority)
- 조직의 전략적 목표와 개인 역할의 연계성 부족
- 변화 관리 프로세스의 투명성 부족
- 의사소통 채널의 효율성 저하

## 2. 권장 조치 사항

### 단기 조치 (1-3개월)
1. 정기적인 타운홀 미팅 개최
2. 1:1 면담을 통한 개인별 우려사항 청취
3. 명확한 변화 관리 로드맵 공유

### 중기 조치 (3-6개월)
1. 맞춤형 교육 프로그램 도입
2. 멘토링 시스템 구축
3. 

In [5]:
# 예시 3: 고객 지원 티켓 처리를 위한 라우팅 워크플로우
# 내용 분석을 기반으로 지원 티켓을 적절한 팀에 라우팅

support_routes = {
    "billing": """귀하는 결제 지원 전문가 입니다. 다음 지침을 따르세요:
    1. 항상 "결제 지원 답변:"으로 시작하세요
    2. 먼저 구체적인 결제 문제를 인지했음을 알리세요
    3. 요금이나 불일치 사항을 명확히 설명하세요
    4. 구체적인 다음 단계를 일정과 함께 나열하세요
    5. 관련이 있다면 결제 옵션으로 마무리하세요
    
    전문적이면서도 친근한 답변을 유지하세요.
    
    Input: """,
    
    "technical": """귀하는 기술 지원 엔지니어입니다. 다음 지침을 따르세요:
    1. 항상 "기술 지원 답변:"으로 시작하세요
    2. 문제 해결을 위한 정확한 단계를 나열하세요
    3. 관련이 있다면 시스템 요구사항을 포함하세요
    4. 일반적인 문제에 대한 대안을 제공하세요
    5. 필요한 경우 상위 단계 이관 경로로 마무리하세요
    
    명확한 번호가 매겨진 단계와 기술적 세부사항을 사용하세요.
    
    Input: """,
    
    "account": """귀하는 계정 보안 전문가입니다. 다음 지침을 따르세요:
    1. 항상 "계정 지원 답변:"으로 시작하세요
    2. 계정 보안과 인증을 우선시하세요
    3. 계정 복구/변경을 위한 명확한 단계를 제공하세요
    4. 보안 팁과 경고를 포함하세요
    5. 해결 시간에 대한 명확한 기대치를 설정하세요
    
    진지하고 보안 중심적인 어조를 유지하세요.
    
    Input: """,
    
    "product": """귀하는 제품 전문가입니다. 다음 지침을 따르세요:
    1. 항상 "제품 지원 답변:"으로 시작하세요
    2. 기능 교육과 모범 사례에 집중하세요
    3. 구체적인 사용 예시를 포함하세요
    4. 관련 문서 섹션에 대한 링크를 제공하세요
    5. 도움이 될 만한 관련 기능을 제안하세요
    
    교육적이고 격려하는 어조를 유지하세요.
    
    Input: """
}

# Test with different support tickets
tickets = [
    """제목: 계정 접속 불가
    메시지: 안녕하세요, 지난 1시간 동안 로그인을 시도했지만 계속 '잘못된 비밀번호' 오류가 발생합니다.
    제가 올바른 비밀번호를 사용하고 있다고 확신합니다. 접속 권한을 다시 받을 수 있도록 도와주시겠습니까? 오늘 업무 종료 전까지
    보고서를 제출해야 해서 긴급합니다.
    - John""",
    
    """제목: 카드에 예상치 못한 청구
    메시지: 안녕하세요, 방금 귀사에서 제 신용카드에 $49.99가 청구된 것을 확인했는데, 제가 알기로는
    $29.99 요금제를 사용 중이었습니다. 이 청구에 대해 설명해 주시고 실수라면 조정해 주실 수 있나요?
    감사합니다,
    Sarah""",
    
    """제목: 데이터 내보내기 방법?
    메시지: 제 프로젝트 데이터를 모두 엑셀로 내보내야 합니다. 문서를 살펴봤지만 일괄 내보내기 방법을
    찾을 수 없습니다. 이것이 가능한가요? 가능하다면, 단계별로 설명해 주시겠습니까?
    감사합니다,
    Mike"""
]

print("지원 티켓 처리 중...\n")
for i, ticket in enumerate(tickets, 1):
    print(f"\n티켓 {i}:")
    print("-" * 40)
    print(ticket)
    print("\n답변:")
    print("-" * 40)
    response = route(ticket, support_routes)
    print(response)

지원 티켓 처리 중...


티켓 1:
----------------------------------------
제목: 계정 접속 불가
    메시지: 안녕하세요, 지난 1시간 동안 로그인을 시도했지만 계속 '잘못된 비밀번호' 오류가 발생합니다.
    제가 올바른 비밀번호를 사용하고 있다고 확신합니다. 접속 권한을 다시 받을 수 있도록 도와주시겠습니까? 오늘 업무 종료 전까지
    보고서를 제출해야 해서 긴급합니다.
    - John

답변:
----------------------------------------

Available routes: ['billing', 'technical', 'account', 'product']
Routing Analysis:

This is clearly an account access issue where the user is unable to log in despite using what they believe is the correct password. The key terms "로그인" (login), "비밀번호" (password), and "계정 접속" (account access) indicate this is an account-related problem. The user also expresses urgency as they need to submit a report before the end of the workday. This type of authentication and access issue falls under the account team's responsibilities, as they handle user credentials and access management.


Selected route: account
계정 지원 답변:

안녕하세요 John님,

계정 접속 문제에 대해 즉시 도움을 드리도록 하겠습니다. 예상 해결 시간은 15-30분입니다.

즉시 조치 사항:
1. 비밀번호 